**Modulo 3: TensorFlow y Keras para Neural Networks**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Parte 3.4: Early Stopping en Keras para Prevenir Overfitting

**Overfitting** ocurre cuando una red neuronal se entrena hasta el punto de que comienza a memorizar en lugar de generalizar, como se muestra en la Figura 3.OVER. 

**Figure 3.OVER: Training vs Validation Error for Overfitting**
![Training vs Validation Error for Overfitting](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_3_training_val.png "Training vs Validation Error for Overfitting")

Es importante segmentar el Dataset original en varios Datasets:

* **Training Set**
* **Validation Set**
* **Holdout Set**

Hay varias formas diferentes de construir estos conjuntos. Los siguientes programas demuestran algunos de estos.

El primer método es un conjunto de entrenamiento y validación. Los datos de entrenamiento se utilizan para entrenar la red neuronal hasta que el conjunto de validación ya no mejora. Esto intenta detenerse en un punto de entrenamiento casi óptimo. Este método solo brindará predicciones precisas "fuera de la muestra" para el conjunto de validación, esto suele ser aproximadamente el 20 % de los datos. Las predicciones para los datos de entrenamiento serán demasiado optimistas, ya que estos fueron los datos con los que se entrenó la red neuronal. La Figura 3.VAL demuestra cómo se divide un conjunto de datos.

**Figure 3.VAL: Training with a Validation Set**
![Training with a Validation Set](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_train_val.png "Training with a Validation Set")

### Early Stopping con clasificación

In [1]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/iris.csv", 
    na_values=['NA', '?'])

# Convertir a numpy - Clasificación
x = df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values
dummies = pd.get_dummies(df['species']) # Clasificación
species = dummies.columns
y = dummies.values

# Dividir en validation y training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Construir la red neural
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Oculta 1
model.add(Dense(25, activation='relu')) # Oculta 2
model.add(Dense(y.shape[1],activation='softmax')) # Salida
model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
        verbose=1, mode='auto', restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
4/4 - 1s - loss: 1.8573 - val_loss: 1.5282 - 929ms/epoch - 232ms/step
Epoch 2/1000
4/4 - 0s - loss: 1.5386 - val_loss: 1.3284 - 29ms/epoch - 7ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.3444 - val_loss: 1.2011 - 30ms/epoch - 7ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.2170 - val_loss: 1.1170 - 29ms/epoch - 7ms/step
Epoch 5/1000
4/4 - 0s - loss: 1.1240 - val_loss: 1.0368 - 32ms/epoch - 8ms/step
Epoch 6/1000
4/4 - 0s - loss: 1.0387 - val_loss: 0.9620 - 29ms/epoch - 7ms/step
Epoch 7/1000
4/4 - 0s - loss: 0.9670 - val_loss: 0.8947 - 31ms/epoch - 8ms/step
Epoch 8/1000
4/4 - 0s - loss: 0.9059 - val_loss: 0.8402 - 30ms/epoch - 7ms/step
Epoch 9/1000
4/4 - 0s - loss: 0.8567 - val_loss: 0.7979 - 30ms/epoch - 7ms/step
Epoch 10/1000
4/4 - 0s - loss: 0.8186 - val_loss: 0.7610 - 32ms/epoch - 8ms/step
Epoch 11/1000
4/4 - 0s - loss: 0.7864 - val_loss: 0.7267 - 30ms/epoch - 8ms/step
Epoch 12/1000
4/4 - 0s - loss: 0.7528 - val_loss: 0.6954 - 30ms/epoch - 8ms/step
Epoch 13/1000
4/4 - 0s - loss: 0.7

Hay una serie de parámetros que se especifican para el objeto **EarlyStopping**. 

* **min_delta** Este valor debe mantenerse pequeño. Simplemente significa el cambio mínimo en error para ser registrado como una mejora. Configurarlo aún más pequeño probablemente no tendrá un gran impacto.
* **patience** ¿Cuánto tiempo debe esperar el entrenamiento para que mejore el error de validación?  
* **verbose** ¿Cuánta información de progreso quieres?
* **mode** En general, establezca siempre esto en "automático". Esto le permite especificar si el error debe minimizarse o maximizarse. Considere la precisión, donde se desean números más altos frente a log-loss/RMSE donde se desean números más bajos.
* **restore_best_weights** Esto siempre debe establecerse en verdadero. Esto restaura los pesos a los valores que tenían cuando el conjunto de validación es el más alto. A menos que usted mismo realice un seguimiento manual de los pesos, debe hacer que Keras realice este paso por usted.

Como puede ver desde arriba, no se utilizó el número total de épocas solicitadas. El entrenamiento de la red neuronal se detuvo una vez que el conjunto de validación dejó de mejorar.

In [2]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 1.0


### Early Stopping con Regresión

El siguiente código demuestra cómo podemos aplicar la detención anticipada a un problema de regresión. La técnica es similar a la detención anticipada del código de clasificación que acabamos de ver.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])

cars = df['name']

# tratado de missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas a Numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regresión

# Dividir validation y training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# construir la red neural
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Oculta 1
model.add(Dense(10, activation='relu')) # Oculta 2
model.add(Dense(1)) # Salida
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

Epoch 1/1000
10/10 - 0s - loss: 908625.8125 - val_loss: 649708.1875 - 329ms/epoch - 33ms/step
Epoch 2/1000
10/10 - 0s - loss: 522234.0938 - val_loss: 353329.7188 - 55ms/epoch - 5ms/step
Epoch 3/1000
10/10 - 0s - loss: 281222.0000 - val_loss: 188052.2188 - 53ms/epoch - 5ms/step
Epoch 4/1000
10/10 - 0s - loss: 147339.6094 - val_loss: 106029.1406 - 57ms/epoch - 6ms/step
Epoch 5/1000
10/10 - 0s - loss: 92525.9531 - val_loss: 71449.0781 - 57ms/epoch - 6ms/step
Epoch 6/1000
10/10 - 0s - loss: 61915.6562 - val_loss: 47527.8398 - 56ms/epoch - 6ms/step
Epoch 7/1000
10/10 - 0s - loss: 41048.1680 - val_loss: 31603.6309 - 54ms/epoch - 5ms/step
Epoch 8/1000
10/10 - 0s - loss: 27357.6367 - val_loss: 20776.6133 - 58ms/epoch - 6ms/step
Epoch 9/1000
10/10 - 0s - loss: 17833.9199 - val_loss: 13464.7178 - 53ms/epoch - 5ms/step
Epoch 10/1000
10/10 - 0s - loss: 11582.2871 - val_loss: 8414.0381 - 51ms/epoch - 5ms/step
Epoch 11/1000
10/10 - 0s - loss: 7232.0874 - val_loss: 5146.3330 - 47ms/epoch - 5ms/step
E

Finalmente, evaluamos el error.

In [5]:
#RMSE error.
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 6.6490729876769255
